## Crypto Clustering

* Import the Data (provided in the starter code)
* Prepare the Data (provided in the starter code)
* Find the Best Value for `k` Using the Original Data
* Cluster Cryptocurrencies with K-means Using the Original Data
* Optimize Clusters with Principal Component Analysis
* Find the Best Value for `k` Using the PCA Data
* Cluster the Cryptocurrencies with K-means Using the PCA Data
* Visualize and Compare the Results

### Import the Data

1. Read  the “crypto_market_data.csv” file from the Resources folder into a DataFrame, and use `index_col="coin_id"` to set the cryptocurrency name as the index. Review the DataFrame.

2. Generate the summary statistics, and use HvPlot to visualize your data to observe what your DataFrame contains.

In [85]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
# hvplot.extension('bokeh')  
from sklearn.cluster import KMeans
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

In [86]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    Path("Resources/crypto_market_data.csv"),
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [87]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [88]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    rot=90,
    width =1000,
    height = 600 
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

1. Use the `StandardScaler` module from scikit-learn to normalize the CSV file data. This will require you to utilize the `fit_transform` function.

2. Create a DataFrame that contains the scaled data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.

In [89]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_num_cols = ['price_change_percentage_24h',
                    'price_change_percentage_7d',
                    'price_change_percentage_14d',
                    'price_change_percentage_30d',
                    'price_change_percentage_60d',
                    'price_change_percentage_200d',
                    'price_change_percentage_1y']

market_data_scaled = StandardScaler().fit_transform(
    df_market_data[market_data_num_cols]
)

In [90]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    market_data_scaled, columns= ['pc_24', 'pc_7d', 'pc_14d', 'pc_30d','pc_60d', 'pc_200d', 'pc_1y']
)

# Copy the crypto names from the original DataFrame
df_market_data_scaled ['coin-id'] = df_market_data.index

# Set the coin_id column as index
df_market_data_scaled = df_market_data_scaled.set_index('coin-id')

# Display the scaled DataFrame
df_market_data_scaled.head()

,pc_24,pc_7d,pc_14d,pc_30d,pc_60d,pc_200d,pc_1y
coin-id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Scaled DataFrame.

1. Code the elbow method algorithm to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following question: What is the best value for `k`?

In [91]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1,11))

In [92]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
    
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
    model=KMeans(n_clusters=k, random_state=42)
# 2. Fit the model to the data using `df_market_data_scaled`
    model.fit(df_market_data_scaled)
# 3. Append the model.inertia_ to the inertia list
    inertia.append(model.inertia_)

In [93]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    'k':k_values, 
    'inertia': inertia
    }

# Create a DataFrame with the data to plot the Elbow curve
df_elbow  = pd.DataFrame(elbow_data)
df_elbow

,k,inertia
0,1,287.000000
1,2,212.123342
2,3,170.193571
3,4,79.022435
4,5,71.097914
5,6,66.108827
6,7,54.488123
7,8,43.224693
8,9,32.860884
9,10,28.404885


In [94]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_plot = df_elbow.hvplot.line(
    x="k",
    y="inertia",
    xticks=k,
    title="Elbow Curve",
    width =1000,
    height = 600 
)
elbow_plot

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:**  The elbow curve above shows the best value for 'K' as 4.

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the original data.

3. Predict the clusters to group the cryptocurrencies using the original data. View the resulting array of cluster values.

4. Create a copy of the original data and add a new column with the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [95]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=42)

In [96]:
# Fit the K-Means model using the scaled DataFrame
model.fit(df_market_data_scaled)

KMeans(n_clusters=4, random_state=42)

In [97]:
# Predict the clusters to group the cryptocurrencies using the scaled data
crypto_clusters = model.predict(df_market_data_scaled)

# Print the resulting array of cluster values.
print(crypto_clusters)

[0 0 2 2 0 0 0 0 0 2 2 2 2 0 2 0 2 2 0 2 2 0 2 2 2 2 2 2 0 2 2 2 3 0 2 2 1
 2 2 2 2]


In [98]:
# Create a copy of the scaled DataFrame
df_market_data_scaled_predictions = df_market_data_scaled.copy()

In [99]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data_scaled_predictions["CryptoCluster"] = crypto_clusters

# Display sample data
df_market_data_scaled_predictions.head()

,pc_24,pc_7d,pc_14d,pc_30d,pc_60d,pc_200d,pc_1y,CryptoCluster
coin-id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,0
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,0
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,2
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,2
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,0


In [100]:
# Create a scatter plot using hvPlot by setting
scatter_plot = df_market_data_scaled_predictions.hvplot.scatter(
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
    x="pc_24",
    y="pc_7d",
# Color the graph points with the labels found using K-Means and
    by="CryptoCluster",
# add the crypto name in the `hover_cols` parameter to identify
    hover_cols="coin_id",
# the cryptocurrency represented by each data point.
    title = "Scatter Plot by Crypto Currency Using Scaled Data",
    width =1000,
    height = 600 
)
scatter_plot

:NdOverlay   [CryptoCluster]
   :Scatter   [pc_24]   (pc_7d)

---

### Optimize Clusters with Principal Component Analysis.

1. Create a PCA model instance and set `n_components=3`.

2. Use the PCA model to reduce to three principal components. View the first five rows of the DataFrame. 

3. Retrieve the explained variance to determine how much information can be attributed to each principal component.

4. Answer the following question: What is the total explained variance of the three principal components?

5. Create a new DataFrame with the PCA data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.

In [101]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3, random_state=42)

In [102]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
market_data_scaled_pca = pca.fit_transform(df_market_data_scaled_predictions)
# View the first five rows of the DataFrame. 
market_data_scaled_pca[:5]

array([[ 1.58513942,  0.13439444,  0.545621  ],
       [ 1.32258066,  0.05449393,  1.11570854],
       [-0.26633536, -0.53640937, -0.79694588],
       [-0.25722513, -0.59201286, -0.6465287 ],
       [ 2.95224985,  0.42237155,  1.26789353]])

In [103]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
display(pca.explained_variance_ratio_)
print(f'Sum of array:{sum(pca.explained_variance_ratio_)}')

array([0.37005408, 0.32322221, 0.19115222])

Sum of array:0.8844285111826466


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 88.45% or 0.8844285111826466 (sum of the explained variances of each individual principle component)

In [104]:
# Create a new DataFrame with the PCA data.

# Creating a DataFrame with the PCA data
df_market_data_pca = pd.DataFrame(market_data_scaled_pca, columns=["PC1", "PC2","PC3"])
# Copy the crypto names from the original data
df_market_data_pca["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_pca = df_market_data_pca.set_index("coin_id")

# Display sample data
df_market_data_pca.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,1.585139,0.134394,0.545621
ethereum,1.322581,0.054494,1.115709
tether,-0.266335,-0.536409,-0.796946
ripple,-0.257225,-0.592013,-0.646529
bitcoin-cash,2.952250,0.422372,1.267894


---

### Find the Best Value for k Using the Scaled PCA DataFrame

1. Code the elbow method algorithm and use the PCA data to find the best value for `k`. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of `k` to visually identify the optimal value for `k`.

3. Answer the following questions: What is the best value for k when using the PCA data? Does it differ from the best k value found using the original data?

In [105]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1, 11))

In [106]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
for i in k:
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
    model = KMeans(n_clusters=i, random_state=42)
# 2. Fit the model to the data using `df_market_data_pca`
    model.fit(df_market_data_pca)
# 3. Append the model.inertia_ to the inertia list
    inertia.append(model.inertia_)

In [107]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {
    "k": k,
    "inertia": inertia
}
# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data_pca)

In [108]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca = df_elbow_pca.hvplot.line(
                                        x="k", 
                                        y="inertia", 
                                        title="Elbow Curve Using PCA Data", 
                                        xticks=k,
                                        width =1000,
                                        height = 600 )
elbow_plot_pca

:Curve   [k]   (inertia)

#### Answer the following questions: 
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for k when using the PCA data is 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** It did not differ from when using the original data in this instance, inertia decreases more slowly from value 4 of K!

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

1. Initialize the K-Means model with four clusters using the best value for `k`. 

2. Fit the K-Means model using the PCA data.

3. Predict the clusters to group the cryptocurrencies using the PCA data. View the resulting array of cluster values.

4. Add a new column to the DataFrame with the PCA data to store the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="PC1"` and `y="PC2"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [109]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=2, random_state=42)

In [110]:
# Fit the K-Means model using the PCA data
model.fit(df_market_data_pca)

KMeans(n_clusters=2, random_state=42)

In [111]:
# Predict the clusters to group the cryptocurrencies using the PCA data
crypto_pca_clusters = model.predict(df_market_data_pca)
# Print the resulting array of cluster values.
print(crypto_pca_clusters)

[0 0 1 1 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0
 1 1 1 1]


In [112]:
# Create a copy of the DataFrame with the PCA data
df_market_pca_predictions = df_market_data_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
df_market_pca_predictions["CryptoClusterPCA"] = crypto_pca_clusters

# Display sample data
df_market_pca_predictions.head()

,PC1,PC2,PC3,CryptoClusterPCA
coin_id,,,,
bitcoin,1.585139,0.134394,0.545621,0
ethereum,1.322581,0.054494,1.115709,0
tether,-0.266335,-0.536409,-0.796946,1
ripple,-0.257225,-0.592013,-0.646529,1
bitcoin-cash,2.952250,0.422372,1.267894,0


In [113]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Colour the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
scatter_plot_pca = df_market_pca_predictions.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="CryptoClusterPCA",
    hover_cols="coin_id",
    title = "Scatter Plot by Crypto Currency Using PCA Data",
    width =1000,
    height = 600 
)
scatter_plot_pca

:NdOverlay   [CryptoClusterPCA]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

1. Create a composite plot using hvPlot and the plus (`*+*`) operator to contrast the Elbow Curve that you created to find the best value for `k` with the original and the PCA data.

2. Create a composite plot using hvPlot and the plus (`+`) operator to contrast the cryptocurrencies clusters using the original and the PCA data.

3. Answer the following question: After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

In [114]:
# Combine into a composite layout
layout_elbow = (elbow_plot * elbow_plot_pca).opts(width=1000, height=600)
layout_elbow

:Overlay
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [115]:
# Set width and height for individual plots
scatter_plot = scatter_plot.opts(width=750, height=600)
scatter_plot_pca = scatter_plot_pca.opts(width=750, height=600)

# Combine into a composite layout
layout_scatter = (scatter_plot + scatter_plot_pca).cols(2)
layout_scatter

:Layout
   .NdOverlay.I  :NdOverlay   [CryptoCluster]
      :Scatter   [pc_24]   (pc_7d)
   .NdOverlay.II :NdOverlay   [CryptoClusterPCA]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The clusters become more distinct and easier to see visually. PCA removes less informative features from the data, as it retains the most significance variance in the original dataset. The second scatterplot above for PCA, shows four distinct clusters, with 'ethland' in orange and 'celsius-degree-token' in yellow. There two cryptocurrencies incidently showed significantly higher percentage price change at 200d and 1 year, compared to all the other cryptocurrencies. Splitting these two out into their own clusters makes sense. Using fewer features forces the blue and red (segment 0 and 1) to be much more closely clustered around the 0 value.